In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Imports

In [6]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split, cross_val_score
import optuna

# Data

In [7]:
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

X = train_data.copy()
X.drop('id', axis=1, inplace=True)
y = train_data["Age"]

no_id_test_data = test_data.drop('id', axis=1) 

display(X.head())
display(y.head())

,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,Age
0,I,1.5250,1.1750,0.3750,28.973189,12.728926,6.647958,8.348928,9
1,I,1.1000,0.8250,0.2750,10.418441,4.521745,2.324659,3.401940,8
2,M,1.3875,1.1125,0.3750,24.777463,11.339800,5.556502,6.662133,9
3,F,1.7000,1.4125,0.5000,50.660556,20.354941,10.991839,14.996885,11
4,I,1.2500,1.0125,0.3375,23.289114,11.977664,4.507570,5.953395,8


0     9
1     8
2     9
3    11
4     8
Name: Age, dtype: int64

# Ordinal Encoding

In [8]:
# # Displays columns
# display(X.columns)

# # Shows that the "Sex" column only has 3 values: "I", "M", and "F"
# display(X['Sex'].unique())

# # Ordinal encoding for the sex column for X
def ordinal_encode_sex(data, inverse=False):

    sexToInt = {
        'I':0,
        'M':1,
        'F':2
    }
    
    intToSex = {}
    
    if(inverse):
        for k, v in sexToInt.items():
            intToSex[v]=k
            
            
    if(not inverse):
        for i in range(0, len(X['Sex'])):
            data[i]=intToSex[data[i]]
    
    else:
        for i in range(0, len(X['Sex'])):
            data[i]=sexToInt[data[i]]
            
    return data
# X['Sex']=[sexToInt[i] for i in X['Sex']]

X['Sex']=ordinal_encode_sex(X['Sex'])
no_id_test_data['Sex']=ordinal_encode_sex(no_id_test_data['Sex'])

KeyError: 'I'

# Hyperparameter Tuning

In [ ]:
def optuna_study(X, y):

    def objective(trial):
        display('hi');
        
        n_estimators = trial.suggest_int('n_estimators', 10, 500)
        max_leaf_nodes = trial.suggest_int('max_leaf_nodes', 1, 100)
        max_depth = trial.suggest_int('max_depth', 1, 30)
        
        rf = RandomForestRegressor(n_estimators=n_estimators,
                        max_leaf_nodes=max_leaf_nodes,
                        max_depth=max_depth)
        
        # # score = StratifiedKFold(rf, X, y)
        display('hello')
        
        # kfold = RepeatedStratifiedKFold(n_splits=40, n_repeats=6, random_state=69);
        
        # scores = []
        
        # display(kfold.split(X, y))
        
        display('hey')
        
        # for train_idx, test_idx in kfold.split(X, y):
            
        #     X_train, X_test = X[train_idx], X[test_idx]
        #     display('elo')
        #     y_train, y_test = y[train_idx], y[test_idx]
        #     display('elo1')
        #     rf.fit(X_train, y_train)
        #     display('elo2')
        #     y_pred = rf.predict(X_test)
        #     scores.append(mean_absolute_error(y_test, y_pred))
        
        # X_train, X_test, y_train, y_test = train_test_split(X, y)
        # rf.fit(X_train, y_train)
        # y_pred = rf.predict(X_test)
        # return mean_absolute_error(y_pred, y_test)
        return cross_val_score(rf, X, y, n_jobs=4, cv=5)
        
        # return np.mean(scores)
        
        display('hehehaha')
        display(scores)
        print(scores)
                        

        # score = cross_val_score(rf, X, y, n_jobs=4, cv=5, scoring='roc_auc')
        return np.mean(scores)

    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=50)
    
    return study.best_params

# Study 1 (no ph + remove outlier + aggregate values, 100 trials): 0.1564147933997004
# Study 2 (no preprocessing, 100 trials): 0.12094767907088448
# Study 3 (preprocessing kudos to MOHAMMAD RAZEGHI, 200 trials): 0.18433600629551264
# Study 4 (same as 3 but max_depth allowed to be 1, 
# {'n_estimators': 31, 'learning_rate': 0.15241724886836167, 'max_depth': 1}): 0.22565032154969472

# *begin using roc_auc

# Study 5 ({'n_estimators': 109, 'learning_rate': 0.03160535667677366, 'max_depth': 1}): 0.7891310192305856
# Study 6 [I 2023-04-15 17:47:02,473] Trial 199 finished with value: 0.756868939982877 and parameters: {'n_estimators': 70, 'learning_rate': 0.21972623175664083, 'max_depth': 2}. Best is trial 196 with value: 0.7902962715736496.
display('hi2')
best_params = optuna_study(X, y)

display(best_params)

'hi2'

[I 2023-06-09 15:39:20,468] A new study created in memory with name: no-name-f3b53cae-3550-46d1-8753-895eb06f2cae


'hi'

'hello'

<generator object _RepeatedSplits.split at 0x000001D2CF5A23E0>

...

[W 2023-06-09 15:43:59,405] Trial 6 failed with parameters: {'n_estimators': 54, 'max_leaf_nodes': 9, 'max_depth': 27} because of the following error: The value array([0.97416531, 0.97420169, 0.97344373, 0.97505068, 0.97563132]) could not be cast to float..
[W 2023-06-09 15:43:59,408] Trial 6 failed with value array([0.97416531, 0.97420169, 0.97344373, 0.97505068, 0.97563132]).


'hi'

'hello'

<generator object _RepeatedSplits.split at 0x000001D2CF5A23E0>

'hey'

# Submission

In [ ]:
final_model = RandomForestRegressor(
    n_estimators = best_params['n_estimators'], 
    max_depth = best_params['max_depth'], 
    learning_rate = best_params['learning_rate'],
    n_jobs=4)

final_model.fit(X, y)
predictions = final_model.predict(no_id_test_data)

submission = pd.DataFrame({'id': test_data.id, 'target': predictions})

display(submission.head())

submission.to_csv(f'submission9.csv', index=False)

print('Successful')